# Load libraries and start chromedriver

In [1]:
#Start the webdriver
import time
from selenium import webdriver
import re
import pandas as pd
from selenium.common.exceptions import NoSuchElementException
from itertools import *
import numpy
import chardet
import numpy as np
import pickle
import itertools

# Scraping

In [2]:
with open("all_links.txt", "rb") as fp:
    links = pickle.load(fp)

In [5]:
path_to_chromedriver = '/Users/isabelbeard/Downloads/chromedriver' # change path as needed
browser = webdriver.Chrome(executable_path = path_to_chromedriver)
#url = 'https://www.skincarisma.com/products/clinique/naturally-gentle-eye-makeup-remover'
#browser.get(url)

In [9]:
%%time

#create empty dataframe
products = pd.DataFrame(columns=['id', 'product', 'brand', 'ratingsCount', 'rating', 'price', 'categories', 'notes'])
ingredients = pd.DataFrame(columns = ['id', 'ingredient', 'function', 'notes', 'ewg', 'cir'])
prodIndex = int(16512)

for link in links[16513:]:
    url = link
    browser.get(url)

    try:
        page = browser.find_element_by_id('main-product')
    
        #get product name
        try:
            product = page.find_element_by_class_name('card-title').text
        except:
            product = 'NaN'

        #get brand
        try:
            brand = page.find_element_by_class_name('card-subtitle').text
        except:
            brand = 'NaN'

        #Get number of ratings
        try:
            rating_group = page.find_element_by_class_name('flex-between')
            num_ratings = rating_group.find_element_by_css_selector('span').text
        except:
            num_ratings = 'NaN'

        #Get rating
        try:
            rating = page.find_element_by_class_name('h6').text
        except:
            rating = 'NaN'

        try:
            all_notes = []
            notes = page.find_elements_by_class_name('p-2')
            for note in notes:
                all_notes.append(note.text)
            all_notes = ', '.join(all_notes)
        except:
            all_notes = 'Nan'

        try:
            categories_div = page.find_elements_by_class_name('category-badge')
            category = []

            for categories in categories_div:
                category.append(categories.text)
            #    print(categories.text)

            category = list(filter(None, category))
            category = ', '.join(category)
        except:
            category = 'NaN'

        try:
            page = browser.find_element_by_id('main-product')
            prices = page.find_elements_by_class_name('currency-section')
            prodPrice = prices[0].text
        except:
            prodPrice = 'Nan'


        one_prod = pd.DataFrame({'id': int(prodIndex), 'product': product, 'price': prodPrice,'brand': brand,'ratingsCount': num_ratings, 'rating': rating, 'notes': all_notes, 'categories': category}, index=[prodIndex])
        products = products.append(one_prod)

        print('id:', prodIndex)
        print('product:', product)
        #print('brand:', brand)
        #print('number of ratings:', num_ratings)
        #print('rating: ', rating)
        #print('notes: ', all_notes)
        #print('category: ', category)
        #print('price: ', prodPrice )

        ##ingredient lists
        try:
            page = browser.find_element_by_id('main-product')
            ingredient_link = page.find_element_by_link_text('VIEW ALL INGREDIENTS')
            ingredient_link.click()
            #print('---ingredient list opened---')
            time.sleep(5)
            page = browser.find_element_by_id('main-product')
            ingredient_table = page.find_element_by_class_name('ingredients-table')
            #print('---ingredient table found---')

            body = ingredient_table.find_element_by_tag_name('tbody')
            rows = body.find_elements_by_tag_name('tr')
            #columns = rows.find_elements_by_tag_name('td')
            #print( '---', len(rows), 'rows found---' )

            num_rows = len(rows)
            num_cols = 4

            for row in rows:
                try:
                    columns = row.find_elements_by_tag_name('td')
                    ewg = columns[0].text
                    cir = columns[1].text
                    ingName = columns[2].text
                    #ingName = ingName.replace("\n", ", ")
                    #ingName = ingName.replace(", ()", "")
                    func = columns[2].find_element_by_tag_name('small').text
                    func = func.replace('(', '')
                    func = func.replace(')', '')
                    notes = columns[3].text
                    #notes = notes.replace("\n\n", ", ")
                    one_ing = pd.DataFrame({'id': prodIndex, 'ingredient_func': ingName, 'function': func,'notes': notes, 'ewg': ewg, 'cir': cir}, index=[row])
                    one_ing['ingredient'] = one_ing['ingredient_func'].replace(to_replace=r'\b'+one_ing['function']+r'\b', value='',regex=True)
                    ##editing of text characters
                    one_ing = one_ing.drop(['ingredient_func'], axis = 1)
                    ingredients = ingredients.append(one_ing)
                except:
                    print('whoops')
                    one_ing = pd.DataFrame({'id': int(prodIndex), 'ingredient': 'NaN', 'function': 'NaN','notes': 'NaN', 'ewg': 'NaN', 'cir': 'NaN'}, index=[row])
                    ingredients = ingredients.append(one_ing)
        except:
            print('whoops')

        ingredients = ingredients.reset_index()
        ingredients = ingredients.drop(['index'], axis = 1)
        products['id'] = products['id'].astype(int)
        ingredients['id'] = ingredients['id'].astype(int)

    except:
        prodIndex = prodIndex + 1
    prodIndex = prodIndex + 1

    time.sleep(5)
    
    

id: 16512
product: Facial Oil
id: 16513
product: Cellularose Micellar Water Cleanser
id: 16514
product: Restore Elixir
id: 16517
product: Repair Me! Reconstructive Hair Treatment
id: 16518
product: Curl Assurance Smoothing Lotion
id: 16519
product: Curl Assurance Smoothing Conditioner
id: 16520
product: Beauticurls Strengthening Hair Conditioner
id: 16521
product: Men Expert Hydra Energetic Comfort
id: 16522
product: BeautiCurls Leave-in Hair Conditioner
id: 16523
product: Beauticurls Argan & Olive Oil Daily Hair Conditioner
id: 16524
product: Super Aqua Ex Jelly Cream
id: 16525
product: White Tone Up Lotion
id: 16526
product: Replenishing Night Cream
id: 16527
product: Revitalizing Day Moisturizer
id: 16528
product: Facial Moisturiser
id: 16529
product: Facial Wash
id: 16530
product: Marvelwhite Zero Blackhead Marvel Gel
id: 16531
product: Cleansing Treatment
id: 16532
product: Natural Mineral Sunscreen Broad Spectrum SPF 30 Body
id: 16533
product: Perfecting Eye Cream
id: 16534
produ

In [10]:
#ingredients.to_csv('ingredients2.csv')
#products.to_csv('products2.csv')

#Wall time: 16h 8min 28s

In [22]:
ing_1 = pd.read_csv('ingredients1.csv', index_col=0)
ing_2 = pd.read_csv('ingredients2.csv', index_col=0)
prod_1 = pd.read_csv('products1.csv', encoding = "ISO-8859-1", index_col=0)
prod_2 = pd.read_csv('products2.csv', encoding = "ISO-8859-1", index_col=0)

print('number of ingredients in df 1: ', len(ing_1))
print('number of ingredients in df 2: ', len(ing_2))
print('number of products in df 1: ', len(prod_1))
print('number of products in df 2: ', len(prod_2))

number of ingredients in df 1:  451636
number of ingredients in df 2:  94004
number of products in df 1:  16512
number of products in df 2:  3399


In [23]:
ingredients_frames = [ing_1, ing_2]
products_frames = [prod_1, prod_2]

ingredients = pd.concat(ingredients_frames)
products = pd.concat(products_frames)

print('number of ingredients: ', len(ingredients))
print('number of products: ', len(products))

number of ingredients:  545640
number of products:  19911


In [26]:
ingredients.to_csv('data/ingredients.csv')
products.to_csv('data/products.csv')